## House listing web-scraper

**Main source:** https://towardsdatascience.com/looking-for-a-house-build-a-web-scraper-to-help-you-5ab25badc83e?gi=9fdc49d7312b

Scraped webpage name and URL is not included

In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import re
import time as t
from numpy import random
import unicodedata
import numpy as np
from datetime import date

### Flat database

In [41]:
def flat_prices(city_list):
    
    FlatDataAll = pd.DataFrame(columns = ['ár (M Ft)', 'nm ár', 'méret', 'cím', 'szobák száma', 'település', 'hirdetés link'])
    
    #pages = list(range(1,2))
    url_fix = "Fix part of the URL"
    
    '''
    all_obs: contains the number of total flat listings in Hungary, this is neccesary to troubleshoot against
    non-existent settlements (if a settlement has no listings the webpage loads with all the listings in Hungary)
    '''
        
    headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
    
    all_obs = int(re.findall(r'\d+', BeautifulSoup(
        get("URL to get the number of pages", headers=headers).text, 
        'html.parser').find_all('div', class_="results__number")[0].text.replace(' ', ''))[0])
    
    #classes = ['price', 'price--sqm', 'listing__address', 'listing__data--area-size']
    
    a = 0
    i = 0
    
    for city in city_list:
        
        link = url_fix + city
        obs = int(re.findall(r'\d+', BeautifulSoup(
            get(link, headers=headers).text, 
            'html.parser').find_all('div', class_="results__number")[0].text.replace(' ', ''))[0])
        
        try:
        
            pages = list(range(1, int(re.findall(
                r'\d+', BeautifulSoup(get(link, headers=headers).text, 'html.parser').find_all(
                    'div', class_="pagination__page-number")[0].text)[1])+1))
        
        except IndexError:
            
            pages = [1]
        
        if (obs < all_obs):
        
            for page in pages:

                url = url_fix + city + '?page=' + str(page)
                response = get(url, headers=headers)
                html_soup = BeautifulSoup(response.text, 'html.parser')

                if len(html_soup.find_all('div', class_="listing__address")) > 0:

                    for obs in range(len(html_soup.find_all('div', class_="listing__address"))):

                        #info_list = []
                        
                        #Av. (scrapable) variables
                        house_price = html_soup.find_all('div', class_="price")[obs].text
                        house_price_sqm = html_soup.find_all('div', class_="price--sqm")[obs].text
                        house_address = html_soup.find_all('div', class_="listing__address")[obs].text
                        house_sqm = html_soup.find_all('div', class_="listing__data--area-size")[obs].text
                        house_rooms =  html_soup.find_all('div', class_="listing__data--room-count")[obs].text

                        url = html_soup.find_all('a', class_="listing__link js-listing-active-area")[obs].get('href')
                        url = "Webpage URL" + url

                        settlement = city

                        info_list = [house_price, house_price_sqm, house_sqm, house_address, house_rooms, settlement, url]

                        info_series = pd.Series(info_list, 
                                                index = ['ár (M Ft)', 'nm ár', 'méret', 'cím', 'szobák száma', 'település', 'hirdetés link'])
                        FlatDataAll = FlatDataAll.append(info_series, ignore_index=True)

                else:

                    break


                i += 1

                #print(str(i) + ' pages are done')

                t.sleep(random.uniform(1,4))
        
        
            a+=1
            print(str(a) + ' cities are done')
            
        else:
            continue
    #Gen int from string vars
    FlatDataAll['ár (M Ft)'] = FlatDataAll['ár (M Ft)'].str.replace(' M Ft', '').astype(float)
    FlatDataAll['nm ár'] = FlatDataAll['nm ár'].str.replace(' ', '')
    FlatDataAll['nm ár'] = FlatDataAll['nm ár'].str.replace('Ft/m2', '').astype(int)
    FlatDataAll['méret'] = FlatDataAll['méret'].str.extract('(\d+)')
    FlatDataAll['telek méret'] = 0
    FlatDataAll['típus'] = 'Lakás' 
    FlatDataAll['Adatszerzés dátuma'] = str(date.today()) 
    
    return FlatDataAll

In [2]:
city_list = list_of_cities  ## All cities and settlements 
flats_data = flat_prices(city_list)

### House database

In [1]:
def house_prices(city_list):
    
    HouseDataAll = pd.DataFrame(columns = ['ár (M Ft)', 'nm ár', 'méret', 'telek méret', 'cím', 'szobák száma', 'település', 'hirdetés link'])
    
    pages = list(range(1,2))
    url_fix = "Fix part of the URL"
    
    '''
    all_obs: contains the number of total flat listings in Hungary, this is neccesary to troubleshoot against
    non-existent settlements (if a settlement has no listings a webpage loads with all the listings in Hungary)
    '''
        
    headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
    
    all_obs = int(re.findall(r'\d+', BeautifulSoup(
        get("URL to get the number of pages", headers=headers).text, 
        'html.parser').find_all('div', class_="results__number")[0].text.replace(' ', ''))[0])
    
    #classes = ['price', 'price--sqm', 'listing__address', 'listing__data--area-size']
    a = 0
    i = 0
    
    for city in city_list:
        
        link = url_fix + city
        obs = int(re.findall(r'\d+', BeautifulSoup(
            get(link, headers=headers).text, 
            'html.parser').find_all('div', class_="results__number")[0].text.replace(' ', ''))[0])
        
        try:
        
            pages = list(range(1, int(re.findall(
                r'\d+', BeautifulSoup(get(link, headers=headers).text, 'html.parser').find_all(
                    'div', class_="pagination__page-number")[0].text)[1])+1))
        
        except IndexError:
            
            pages = [1]
            
        if (obs < all_obs):
        
            for page in pages:

                url = url_fix + city + '?page=' + str(page)
                response = get(url, headers=headers)
                html_soup = BeautifulSoup(response.text, 'html.parser')

                if len(html_soup.find_all('div', class_="listing__address")) > 0:

                    for obs in range(len(html_soup.find_all('div', class_="listing__address"))):

                        #info_list = []

                        house_price = html_soup.find_all('div', class_="price")[obs].text
                        house_price_sqm = html_soup.find_all('div', class_="price--sqm")[obs].text
                        house_address = html_soup.find_all('div', class_="listing__address")[obs].text
                        house_sqm = html_soup.find_all('div', class_="listing__data--area-size")[obs].text
                        house_plot_size = html_soup.find_all('div', class_="listing__data--plot-size")[obs].text
                        house_rooms =  html_soup.find_all('div', class_="listing__data--room-count")[obs].text
                        
                        url = html_soup.find_all('a', class_="listing__link js-listing-active-area")[obs].get('href')
                        url = "Webpage URL" + url

                        settlement = city

                        info_list = [house_price, house_price_sqm, house_sqm, house_plot_size, house_address, house_rooms, settlement, url]

                        info_series = pd.Series(
                            info_list, index = ['ár (M Ft)', 'nm ár', 'méret', 'telek méret', 'cím', 'szobák száma', 'település', 'hirdetés link'])

                        HouseDataAll = HouseDataAll.append(info_series, ignore_index=True)

                else:

                    break

                i += 1
                print(str(i) + ' pages are done')
                t.sleep(random.uniform(4, 10))


            a+=1
            print(str(a) + ' cities are done')
        
        else:
            continue

    HouseDataAll['ár (M Ft)'] = HouseDataAll['ár (M Ft)'].str.replace(' M Ft', '').astype(float)
    HouseDataAll['nm ár'] = HouseDataAll['nm ár'].str.replace(' ', '')
    HouseDataAll['nm ár'] = HouseDataAll['nm ár'].str.replace('Ft/m2', '').astype(int)
    HouseDataAll['méret'] = HouseDataAll['méret'].str.extract('(\d+)')
    HouseDataAll['telek méret'] = HouseDataAll['telek méret'].str.replace(' ', '')
    HouseDataAll['telek méret'] = HouseDataAll['telek méret'].str.extract('(\d+)')
    HouseDataAll['típus'] = 'Ház' 
    HouseDataAll['Adatszerzés dátuma'] = str(date.today())
    
    return HouseDataAll

In [3]:
city_list = list_of_cities  ## All cities and settlements 
houses_data = house_prices(city_list)

### Merge

In [56]:
DataAll = houses_data.append(flats_data,ignore_index = True)